In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3517295513685306844
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 5435179395996323824
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 17054769935565745035
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7304675328
locality {
  bus_id: 1
  links {
  }
}
incarnation: 42228299961805561
physical_device_desc: "device: 0, name: Tesla P4, pci bus id: 0000:00:04.0, compute capability: 6.1"
]


In [0]:
# import pandas as pd
# import numpy as np
# import tensorflow as tf

# import os.path
# import pickle
# from keras.preprocessing.text import Tokenizer, one_hot
# from keras.preprocessing.sequence import pad_sequences
# from keras.models import Sequential

# from keras.layers.core import Activation, Dropout, Dense
# from keras.layers import Flatten, LSTM
# from keras.layers import GlobalMaxPooling1D
# from keras.models import Model
# from keras.layers.normalization import BatchNormalization

# from sklearn.model_selection import train_test_split
# from sklearn.metrics import precision_score, recall_score, f1_score
# from keras.utils.np_utils import to_categorical
# from keras.callbacks import EarlyStopping
# from keras.layers import Dropout, Input, Embedding
# from keras.layers.merge import Concatenate

# import matplotlib.pyplot as plt
# import nltk
# import re
# import seaborn as sns

# from tqdm import tqdm
# from sklearn.model_selection import train_test_split
# from nltk.tokenize import word_tokenize
# from nltk import pos_tag
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
# import nltk

# from numpy import array
# from numpy import asarray
# from numpy import zeros

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
import nltk
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

In [0]:
real = pd.read_csv('/content/gdrive/My Drive/twitter misinfo/2020_04_10000_real_tweets.csv', header = None)
fake = pd.read_csv('/content/gdrive/My Drive/twitter misinfo/2020_04_10000_fake_tweets_unconditional.csv', header = None)


In [0]:
real.columns = ['tweet']

In [0]:
real['index'] = real.index

In [0]:
fake.columns = ['tweet']
fake['index'] = fake.index

In [0]:
cols = real.columns.tolist()
cols = cols[-1:] + cols[:-1]

In [0]:
real = real[cols]

In [0]:
cols = fake.columns.tolist()
cols = cols[-1:] + cols[:-1]
fake = fake[cols]

In [11]:
real['label'] = 1
fake['label'] = 0
real.head()

,index,tweet,label
0,0,#Taiwan isn't a member of the #WHO. Taiwan tr...,1
1,1,From the DJs to the CJs.. Look whose in the PJ...,1
2,2,The @ClinicalBioinfo led by @xdopazo releases ...,1
3,3,IMF expects #Greece to be the worst hit econom...,1
4,4,More on #COVID19 - China witnessed a slight in...,1


In [12]:
fake.head()

,index,tweet,label
0,0,@chrislhayes I think it's important to note th...,0
1,1,"The problem is, it's the Chinese that are bein...",0
2,2,Do you know of any people who are looking for ...,0
3,3,Just as the COVID-19 pandemic has shown that t...,0
4,4,Covid-19's impact on the economy: The RBC Stat...,0


In [13]:
fake['tweet'][0]

"@chrislhayes I think it's important to note that there are two big areas that need to be addressed. First, the coronavirus and the existing system. Second, the existing system and health systems.  The first is very, very important, and that's a big concern. "

In [14]:
combined = pd.concat([fake, real])
features = combined['tweet']
labels = combined['label']
combined = combined.sample(frac = 1).reset_index(drop = True)
combined['index'] = combined.index
combined.tail()

,index,tweet,label
19992,19992,@MSANNCOOPER @jon_mumford1982 If the selling o...,1
19993,19993,A fund being put together to care for our heal...,1
19994,19994,".@teampentest360, a fast-growing cybersecurity...",1
19995,19995,Thank you @OpenHearings for your hard work and...,0
19996,19996,#FightWithChina The United States and China h...,0


In [0]:
combined['tweet'] = combined['tweet'].astype(str).str.replace('[^a-zA-Z]',' ').str.lower()

In [0]:
X_body_text = combined['tweet'].values
y = combined['label'].values

In [17]:
X_body_text

array(['one of the ways you know that arkansas is under testing is that the border counties in ok  tx  la  mo  and tn  all  show higher rates of coronavirus than ar  not performing enough tests is a good way to keep the numbers down ',
       'i know it s a lot to ask for when we re all dealing with an epidemic  but it s not going to be easy for us to stay calm when we re not in the middle of a war    this is a scary prospect ',
       'spoke to another nurse this week who was also hearing about trash bags as ppe  she said maybe the protection was good  but look at this image and what it says about what we think of nurses ',
       ...,
       '  teampentest     a fast growing cybersecurity startup in bahrain has committed to providing smes in the kingdom with various free services for three months in the wake of the  covid   crisis ',
       'thank you  openhearings for your hard work and support  we are facing a lot of stress due to  covid    we want to show how we are doing our jobs

In [18]:
y

array([1, 0, 1, ..., 1, 0, 0])

In [0]:
from sklearn.naive_bayes import MultinomialNB

nb_body = MultinomialNB()

In [0]:
# def mod(X_body_text, nb_body):
#     upperLimitRange = range(2,20)
#     max_df_range = np.linspace(0.2,0.9,71)
#     max_f1_train = 0
#     max_accuracy_train = 0
#     max_f1_acc_train = 0
#     max_f1_test = 0
#     max_accuracy_test = 0
#     max_f1_acc_test = 0
#     best_ngram = 0
#     best_max_df = 0
#     count = 0
#     for i in upperLimitRange:
#         for j in max_df_range:
#             tfidf = TfidfVectorizer(ngram_range=(1,i), max_df = j, min_df = 0.01)
#             X_body_tfidf = tfidf.fit_transform(X_body_text)
#             indices = combined.index.values
#             X_body_tfidf_train, X_body_tfidf_test, y_body_train, y_body_test, indices_body_train, indices_body_test = train_test_split(X_body_tfidf, y, indices, test_size = 0.2, random_state=42)
#             nb_body.fit(X_body_tfidf_train, y_body_train)
#             y_body_train_pred = nb_body.predict(X_body_tfidf_train)

#             f1_score_train = f1_score(y_body_train, y_body_train_pred, average='macro')
#             accuracy_score_train = accuracy_score(y_body_train, y_body_train_pred)

#             if f1_score_train > max_f1_train:
#                 max_f1_train = f1_score_train
#                 max_f1_acc_train = accuracy_score_train
#                 best_ngram = i
#                 best_max_df = j

#             if accuracy_score_train > max_accuracy_train:
#                 max_accuracy_train = accuracy_score_train
#                 print(max_accuracy_train)
            
#             count = count + 1
#             print(count)

#             # y_body_pred = nb_body.predict(X_body_tfidf_test)
            
#             # f1_score_test = f1_score(y_body_test, y_body_pred, average='macro')
#             # accuracy_score_test = accuracy_score(y_body_test, y_body_pred)

#             # if f1_score_test > max_f1_test:
#             #     max_f1_test = f1_score_test
#             #     max_f1_acc_test = accuracy_score_test
#             #     best_ngram = i
#             #     best_max-df = j
            
#             # if accuracy_score_test > max_accuracy_test:
#             #     max_accuracy_test = accuracy_score_test
#             #     print(max_accuracy_test)

#     return best_ngram, best_max_df, max_f1_train, max_f1_acc_train


# best_ngram, best_max_df, max_f1_train, max_f1_acc_train = mod(X_body_text, nb_body)
# print("Best ngram value = ", best_ngram)
# print("Best max_df_value = ", best_max_df)
# print("Best training f1_score = ", max_f1_train)
# print("Accuracy for the best f1_score = ", max_f1_acc_train)





In [0]:
best_ngram = 2
best_max_df = 0.62


In [0]:
tfidf = TfidfVectorizer(ngram_range=(1,best_ngram), max_df= best_max_df, min_df= 0.01)

In [0]:
X_body_tfidf = tfidf.fit_transform(X_body_text)

In [0]:
indices = combined.index.values

In [0]:
X_body_tfidf_train, X_body_tfidf_test, y_body_train, y_body_test, indices_body_train, indices_body_test = train_test_split(X_body_tfidf, y, indices, test_size = 0.2, random_state=42)

In [26]:
combined.loc[indices_body_train].groupby('label').agg('count')

,index,tweet
label,,
0,7980,7980
1,8017,8017


In [27]:
combined.loc[indices_body_test].groupby('label').agg('count')

,index,tweet
label,,
0,2017,2017
1,1983,1983


In [0]:
from sklearn.naive_bayes import MultinomialNB

In [0]:
nb_body = MultinomialNB()

In [30]:
nb_body.fit(X_body_tfidf_train, y_body_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
y_body_train_pred = nb_body.predict(X_body_tfidf_train)

In [32]:
print('Naive Bayes Train - F1 and Accuracy Scores:')
print('F1 score {:.4}%'.format(f1_score(y_body_train, y_body_train_pred, average='macro')*100 ))
print('Accuracy score {:.4}%'.format(accuracy_score(y_body_train, y_body_train_pred)*100))

Naive Bayes Train - F1 and Accuracy Scores:
F1 score 72.48%
Accuracy score 72.49%


In [33]:
np.where(y_body_train != y_body_train_pred)

(array([    0,     1,     7, ..., 15981, 15983, 15989]),)

In [34]:
combined.loc[[52, 449]]

,index,tweet,label
52,52,i sat down with salliekrawcheck of ellevest ...,1
449,449,restorations of lical bodies with full strengt...,1


In [0]:
y_body_pred = nb_body.predict(X_body_tfidf_test)

In [38]:
print('Naive Bayes Test - F1 and Accuracy Scores: (Unconditional)')
print('F1 score {:.4}%'.format(f1_score(y_body_test, y_body_pred, average='macro')*100 ))
print('Accuracy score {:.4}%'.format(accuracy_score(y_body_test, y_body_pred)*100))

Naive Bayes Test - F1 and Accuracy Scores: (Unconditional)
F1 score 72.33%
Accuracy score 72.35%


In [37]:
np.where(y_body_test != y_body_pred)

(array([   6,    8,    9, ..., 3996, 3998, 3999]),)